create_genre_dataset.py
   by T. Bertin-Mahieux (2011) Columbia University
      tb2332@columbia.edu

USAGE
   ./create_genre_dataset.py <msd dir> <tm.db> <artist_tag.db> <output>
PARAMS
         msd dir   - main directory "/data" of the MSD
           tm.db   - track_metadata.db
   artist_tag.db   - SQLite db with terms and tags
          output   - output dataset in a text file


SystemExit: 0

In [ ]:
import pandas as pd

dataframeArtist = pd.read_csv('artists.csv', header=0)
del dataframeArtist['year_of_birth']
del dataframeArtist['country']
del dataframeArtist['city']
del dataframeArtist['email']
del dataframeArtist['zip_code']
dataframeAlbum = pd.read_csv('albums.csv', header=0)
del dataframeAlbum['year_of_pub']
del dataframeAlbum['num_of_tracks']
del dataframeAlbum['num_of_sales']
del dataframeAlbum['rolling_stone_critic']
del dataframeAlbum['mtv_critic']
del dataframeAlbum['music_maniac_critic']

print(dataframeArtist)


In [ ]:
print(dataframeAlbum)

In [ ]:
def getArtistInfo(artistId, artistDataframe):
    return artistDataframe.iloc[[artistId - 1]]

def getAlbums(artistId, albumDataframe):
    return albumDataframe.loc[albumDataframe['artist_id'] == artistId]



print(getArtistInfo(1767, dataframeArtist))
print(getAlbums(1767, dataframeAlbum))